# Chicago Car Crashes

Define your business problem
Define your stakeholder
What is your target variable
What is your first simple model and WHY
What are your hurdles thus far
How are you planning out the rest of the week? 

In [1]:
import numpy as np
import pandas as pd
import csv
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
vehicle_df = pd.read_csv('Data/Traffic_Crashes_-_Vehicles.csv', low_memory=False)

In [3]:
crash_df = pd.read_csv('Data/Traffic_Crashes_-_Crashes.csv', low_memory=False)

In [4]:
people_df = pd.read_csv('Data/Traffic_Crashes_-_People.csv', low_memory=False)

# People DF 

In [5]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226112 entries, 0 to 1226111
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1226112 non-null  object 
 1   PERSON_TYPE            1226112 non-null  object 
 2   CRASH_RECORD_ID        1226112 non-null  object 
 3   RD_NO                  1216612 non-null  object 
 4   VEHICLE_ID             1201828 non-null  float64
 5   CRASH_DATE             1226112 non-null  object 
 6   SEAT_NO                250405 non-null   float64
 7   CITY                   901198 non-null   object 
 8   STATE                  912037 non-null   object 
 9   ZIPCODE                822798 non-null   object 
 10  SEX                    1207581 non-null  object 
 11  AGE                    873496 non-null   float64
 12  DRIVERS_LICENSE_STATE  723058 non-null   object 
 13  DRIVERS_LICENSE_CLASS  621947 non-null   object 
 14  SAFETY_EQUIPMENT  

In [6]:
people_df['PHYSICAL_CONDITION'].value_counts()

NORMAL                          649211
UNKNOWN                         306266
IMPAIRED - ALCOHOL                4656
REMOVED BY EMS                    3778
OTHER                             2818
FATIGUED/ASLEEP                   2600
EMOTIONAL                         2136
ILLNESS/FAINTED                    917
HAD BEEN DRINKING                  771
IMPAIRED - DRUGS                   564
IMPAIRED - ALCOHOL AND DRUGS       236
MEDICATED                          123
Name: PHYSICAL_CONDITION, dtype: int64

In [52]:
people_drivers_df = people_df[people_df['PERSON_TYPE'] == 'DRIVER']
people_drivers_df

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225947,O1207013,DRIVER,23480f2833dee6b2830f61c35d88660c32261a08a048cc...,JE411672,1145289.0,10/08/2021 06:56:00 PM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1225948,O1209234,DRIVER,5ed9bd705662c986ffc38ea3d99d5e9f86c39fb2d2cb6c...,JE402900,1147337.0,10/08/2021 04:00:00 PM,NaN,WILMETTE,IL,60091,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1225949,O1209235,DRIVER,5ed9bd705662c986ffc38ea3d99d5e9f86c39fb2d2cb6c...,JE402900,1147338.0,10/08/2021 04:00:00 PM,NaN,CHICAGO,IL,60622,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1225950,O1210146,DRIVER,cb59c5ae61040276690e3e008006cc962d5a3099e5a18a...,JE404123,1148220.0,10/08/2021 08:30:00 PM,NaN,CHICAGO,IL,60643,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [58]:
people_drivers_df['AGE'].head(50)

0     25.0
1     37.0
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12    34.0
13    53.0
14    65.0
15     NaN
16    26.0
17    64.0
18    33.0
19    22.0
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
31     NaN
32     NaN
33     NaN
34     NaN
35     NaN
36    70.0
37    51.0
38    25.0
39    22.0
40    61.0
41     NaN
42     NaN
43     NaN
44     NaN
45     NaN
46     NaN
47     NaN
52     NaN
53    48.0
Name: AGE, dtype: float64

##### People df dropping columns

In [7]:
people_df_clean = people_df[["CRASH_RECORD_ID",'CRASH_DATE','SEX', "AGE", "DRIVERS_LICENSE_STATE", "SAFETY_EQUIPMENT",'INJURY_CLASSIFICATION', "DRIVER_ACTION", 
                                      "DRIVER_VISION",'PHYSICAL_CONDITION', "PEDPEDAL_ACTION", "PEDPEDAL_VISIBILITY", "PEDPEDAL_LOCATION", 
                                      "BAC_RESULT", "BAC_RESULT VALUE", "CELL_PHONE_USE"]]



people_df_clean.head()

,CRASH_RECORD_ID,CRASH_DATE,SEX,AGE,DRIVERS_LICENSE_STATE,SAFETY_EQUIPMENT,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,M,25.0,IL,NONE PRESENT,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,M,37.0,IL,SAFETY BELT USED,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,11/01/2015 05:00:00 AM,X,NaN,NaN,USAGE UNKNOWN,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,11/01/2015 08:00:00 AM,X,NaN,NaN,USAGE UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,11/01/2015 10:15:00 AM,X,NaN,NaN,USAGE UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


# VEHICLE_df

In [8]:
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134612 entries, 0 to 1134611
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1134612 non-null  int64  
 1   CRASH_RECORD_ID           1134612 non-null  object 
 2   RD_NO                     1125395 non-null  object 
 3   CRASH_DATE                1134612 non-null  object 
 4   UNIT_NO                   1134612 non-null  int64  
 5   UNIT_TYPE                 1132985 non-null  object 
 6   NUM_PASSENGERS            169810 non-null   float64
 7   VEHICLE_ID                1108583 non-null  float64
 8   CMRC_VEH_I                20992 non-null    object 
 9   MAKE                      1108578 non-null  object 
 10  MODEL                     1108435 non-null  object 
 11  LIC_PLATE_STATE           1011371 non-null  object 
 12  VEHICLE_YEAR              928020 non-null   float64
 13  VEHICLE_DEFECT            1

In [9]:
vehicle_df['GVWR'].value_counts()

3           2052
80000       1188
2            911
1            641
26000        211
            ... 
42,540LG       1
15595          1
140780         1
41000          1
17872          1
Name: GVWR, Length: 421, dtype: int64

In [10]:
vehicle_df_clean = vehicle_df[["CRASH_RECORD_ID",'CRASH_DATE','LIC_PLATE_STATE', "VEHICLE_YEAR","NUM_PASSENGERS", "MANEUVER", "EXCEED_SPEED_LIMIT_I", "VEHICLE_TYPE"]]
vehicle_df_clean.head()

,CRASH_RECORD_ID,CRASH_DATE,LIC_PLATE_STATE,VEHICLE_YEAR,NUM_PASSENGERS,MANEUVER,EXCEED_SPEED_LIMIT_I,VEHICLE_TYPE
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,01/22/2020 06:25:00 AM,IL,2017.0,NaN,STRAIGHT AHEAD,NaN,PASSENGER
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,IL,2016.0,NaN,STRAIGHT AHEAD,NaN,PASSENGER
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,IL,2010.0,NaN,PARKED,NaN,UNKNOWN/NA
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,IL,2008.0,NaN,PARKED,NaN,UNKNOWN/NA
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,IL,NaN,NaN,STRAIGHT AHEAD,NaN,PASSENGER


## CRASH_df

In [11]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554228 entries, 0 to 554227
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                554228 non-null  object 
 1   RD_NO                          549766 non-null  object 
 2   CRASH_DATE_EST_I               41950 non-null   object 
 3   CRASH_DATE                     554228 non-null  object 
 4   POSTED_SPEED_LIMIT             554228 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         554228 non-null  object 
 6   DEVICE_CONDITION               554228 non-null  object 
 7   WEATHER_CONDITION              554228 non-null  object 
 8   LIGHTING_CONDITION             554228 non-null  object 
 9   FIRST_CRASH_TYPE               554228 non-null  object 
 10  TRAFFICWAY_TYPE                554228 non-null  object 
 11  LANE_CNT                       198970 non-null  float64
 12  ALIGNMENT                     

In [12]:
crash_df['LOCATION'].value_counts()

POINT (-87.905309125103 41.976201139024)    747
POINT (-87.619928173678 41.900958919109)    508
POINT (-87.585971992965 41.751460603167)    397
POINT (-87.580147768689 41.791420282098)    378
POINT (-87.585275565077 41.722257273006)    307
                                           ... 
POINT (-87.705132239681 41.972009924192)      1
POINT (-87.735320839547 41.872059423521)      1
POINT (-87.83650104886 41.951166412549)       1
POINT (-87.617372159655 41.775921165248)      1
POINT (-87.780162317876 41.997302747477)      1
Name: LOCATION, Length: 232293, dtype: int64

In [13]:
crash_df_clean = crash_df.drop(columns=["RD_NO", "CRASH_DATE_EST_I", 
                                   "REPORT_TYPE", "HIT_AND_RUN_I", 
                                   "DATE_POLICE_NOTIFIED", "STREET_NO", "BEAT_OF_OCCURRENCE",
                                   "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", 
                                   "MOST_SEVERE_INJURY", "INJURIES_FATAL", "INJURIES_INCAPACITATING", 
                                   "INJURIES_NON_INCAPACITATING", "INJURIES_REPORTED_NOT_EVIDENT", 
                                   "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN"])
crash_df_clean.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,INJURIES_TOTAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NaN,NaN,2.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,...,NaN,NaN,2.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,NaN,...,NaN,NaN,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NaN,NaN,2.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRIVEWAY,NaN,...,NaN,NaN,2.0,0.0,14,4,7,NaN,NaN,NaN


# MERGING THE FIRST 2 DATAFRAMES

#### crash df with vehicle df

In [14]:
crash_df_clean['CRASH_RECORD_ID'].is_unique

True

In [15]:
crash_vehicle_df = pd.merge(crash_df_clean, vehicle_df_clean, how = 'left',  left_on='CRASH_RECORD_ID', right_on='CRASH_RECORD_ID')
crash_vehicle_df

,CRASH_RECORD_ID,CRASH_DATE_x,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,LATITUDE,LONGITUDE,LOCATION,CRASH_DATE_y,LIC_PLATE_STATE,VEHICLE_YEAR,NUM_PASSENGERS,MANEUVER,EXCEED_SPEED_LIMIT_I,VEHICLE_TYPE
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993),07/10/2019 05:56:00 PM,IL,1995.0,NaN,BACKING,NaN,PASSENGER
1,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993),07/10/2019 05:56:00 PM,IL,2012.0,1.0,TURNING RIGHT,NaN,PASSENGER
2,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,...,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989),06/30/2017 04:00:00 PM,IL,2011.0,NaN,TURNING LEFT,NaN,SPORT UTILITY VEHICLE (SUV)
3,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,...,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989),06/30/2017 04:00:00 PM,IL,2016.0,1.0,STRAIGHT AHEAD,NaN,PASSENGER
4,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,NaN,...,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008),07/10/2020 10:25:00 AM,IL,2012.0,NaN,STRAIGHT AHEAD,NaN,PASSENGER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134608,d51aae396db49981c7ee26ceb54dfcab3c4b06d0cc5d7d...,07/12/2021 10:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR TO SIDE,ONE-WAY,NaN,...,41.752918,-87.621573,POINT (-87.621572939578 41.752917594949),07/12/2021 10:00:00 AM,IL,2013.0,NaN,STRAIGHT AHEAD,NaN,PASSENGER
1134609,6f9abc7e7f54095cef0fa17e16e6f72eb14f8d17d3b572...,07/13/2021 04:15:00 PM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,41.781830,-87.668979,POINT (-87.668979124866 41.781829602685),07/13/2021 04:15:00 PM,NaN,NaN,NaN,UNKNOWN/NA,NaN,UNKNOWN/NA
1134610,6f9abc7e7f54095cef0fa17e16e6f72eb14f8d17d3b572...,07/13/2021 04:15:00 PM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,41.781830,-87.668979,POINT (-87.668979124866 41.781829602685),07/13/2021 04:15:00 PM,IL,2017.0,NaN,PARKED,NaN,SPORT UTILITY VEHICLE (SUV)
1134611,bc361965fe506c50f7899493c79b1f7d011376f753acaa...,07/13/2021 11:30:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,41.997303,-87.780162,POINT (-87.780162317876 41.997302747477),07/13/2021 11:30:00 AM,NaN,NaN,NaN,CHANGING LANES,NaN,UNKNOWN/NA


In [16]:
vehicle_df_clean['CRASH_RECORD_ID'].is_unique

False

## Now merge with the third df (People df)

In [17]:
crash_vehicle_df['CRASH_RECORD_ID'].is_unique, people_df_clean['CRASH_RECORD_ID'].is_unique

(False, False)

In [18]:
crash_full = crash_vehicle_df.merge(people_df_clean,how = 'left', left_on=['CRASH_RECORD_ID'], right_on=['CRASH_RECORD_ID'])
crash_full

,CRASH_RECORD_ID,CRASH_DATE_x,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595805,6f9abc7e7f54095cef0fa17e16e6f72eb14f8d17d3b572...,07/13/2021 04:15:00 PM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2595806,bc361965fe506c50f7899493c79b1f7d011376f753acaa...,07/13/2021 11:30:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2595807,bc361965fe506c50f7899493c79b1f7d011376f753acaa...,07/13/2021 11:30:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2595808,bc361965fe506c50f7899493c79b1f7d011376f753acaa...,07/13/2021 11:30:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [19]:
crash_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2595810 entries, 0 to 2595809
Data columns (total 55 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE_x             object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   LANE_CNT                 float64
 10  ALIGNMENT                object 
 11  ROADWAY_SURFACE_COND     object 
 12  ROAD_DEFECT              object 
 13  CRASH_TYPE               object 
 14  INTERSECTION_RELATED_I   object 
 15  NOT_RIGHT_OF_WAY_I       object 
 16  DAMAGE                   object 
 17  PRIM_CONTRIBUTORY_CAUSE  object 
 18  SEC_CONTRIBUTORY_CAUSE   object 
 19  STREET_DIRECTION         object 
 20  STREET_NAME              object 
 21  DOORING_

#### drop some columns to match Lenore's 

In [20]:
crash_full.drop(columns=['CRASH_DATE_x','CRASH_DATE','SEX','DRIVERS_LICENSE_STATE','CRASH_DATE_y','PHYSICAL_CONDITION'], inplace=True)

In [21]:
crash_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2595810 entries, 0 to 2595809
Data columns (total 49 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   POSTED_SPEED_LIMIT       int64  
 2   TRAFFIC_CONTROL_DEVICE   object 
 3   DEVICE_CONDITION         object 
 4   WEATHER_CONDITION        object 
 5   LIGHTING_CONDITION       object 
 6   FIRST_CRASH_TYPE         object 
 7   TRAFFICWAY_TYPE          object 
 8   LANE_CNT                 float64
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  CRASH_TYPE               object 
 13  INTERSECTION_RELATED_I   object 
 14  NOT_RIGHT_OF_WAY_I       object 
 15  DAMAGE                   object 
 16  PRIM_CONTRIBUTORY_CAUSE  object 
 17  SEC_CONTRIBUTORY_CAUSE   object 
 18  STREET_DIRECTION         object 
 19  STREET_NAME              object 
 20  DOORING_I                object 
 21  WORK_ZON

# DATA CLEANING

In [22]:
crash_full['MANEUVER'].value_counts()

STRAIGHT AHEAD                        1285276
SLOW/STOP IN TRAFFIC                   241663
PARKED                                 202802
TURNING LEFT                           166705
UNKNOWN/NA                             135154
BACKING                                 93120
TURNING RIGHT                           88015
PASSING/OVERTAKING                      65242
CHANGING LANES                          55543
OTHER                                   43197
ENTERING TRAFFIC LANE FROM PARKING      31142
MERGING                                 19154
STARTING IN TRAFFIC                     17393
U-TURN                                  15539
ENTER FROM DRIVE/ALLEY                  10972
AVOIDING VEHICLES/OBJECTS               10750
LEAVING TRAFFIC LANE TO PARK            10445
SKIDDING/CONTROL LOSS                    9754
SLOW/STOP - LEFT TURN                    8126
PARKED IN TRAFFIC LANE                   5444
SLOW/STOP - RIGHT TURN                   4761
SLOW/STOP - LOAD/UNLOAD           

In [23]:
crash_full['EXCEED_SPEED_LIMIT_I'].value_counts()

Y    4570
N    1553
Name: EXCEED_SPEED_LIMIT_I, dtype: int64

In [26]:
crash_full['EXCEED_SPEED_LIMIT_I']=crash_full['EXCEED_SPEED_LIMIT_I'].replace({N:False,Y:True})

NameError: name 'N' is not defined

In [29]:
crash_full['LOCATION'].value_counts()

POINT (-87.905309125103 41.976201139024)    2996
POINT (-87.585971992965 41.751460603167)    2287
POINT (-87.580147768689 41.791420282098)    1943
POINT (-87.619928173678 41.900958919109)    1701
POINT (-87.585275565077 41.722257273006)    1560
                                            ... 
POINT (-87.712521082009 41.793267256395)       1
POINT (-87.736443193166 41.782055228637)       1
POINT (-87.58519153283 41.716781569157)        1
POINT (-87.688264714985 41.757374338318)       1
POINT (-87.676170080728 41.856855055444)       1
Name: LOCATION, Length: 232293, dtype: int64

In [32]:
crash_full.isnull().sum().sort_values()

CRASH_RECORD_ID                  0
DAMAGE                           0
CRASH_HOUR                       0
CRASH_DAY_OF_WEEK                0
CRASH_TYPE                       0
ROAD_DEFECT                      0
ROADWAY_SURFACE_COND             0
ALIGNMENT                        0
CRASH_MONTH                      0
TRAFFICWAY_TYPE                  0
FIRST_CRASH_TYPE                 0
LIGHTING_CONDITION               0
WEATHER_CONDITION                0
DEVICE_CONDITION                 0
TRAFFIC_CONTROL_DEVICE           0
POSTED_SPEED_LIMIT               0
PRIM_CONTRIBUTORY_CAUSE          0
SEC_CONTRIBUTORY_CAUSE           0
NUM_UNITS                        2
STREET_NAME                      4
STREET_DIRECTION                14
INJURIES_TOTAL                1942
INJURY_CLASSIFICATION         3394
SAFETY_EQUIPMENT             10247
LOCATION                     14895
LONGITUDE                    14895
LATITUDE                     14895
VEHICLE_TYPE                 61659
MANEUVER            

In [44]:
people_df['AGE'].value_counts().tail(50)

 11.0     2668
 9.0      2665
 5.0      2663
 8.0      2662
 74.0     2638
 6.0      2623
 10.0     2599
 7.0      2540
 12.0     2513
 75.0     2384
 76.0     2199
 77.0     1948
 78.0     1748
 79.0     1512
 80.0     1381
 81.0     1207
 82.0     1013
 83.0      901
 84.0      757
 85.0      604
 86.0      486
 87.0      455
 88.0      346
 89.0      278
 90.0      235
 91.0      168
 92.0      122
 93.0       94
 94.0       70
 95.0       54
 98.0       27
 97.0       24
 96.0       24
 99.0       18
 100.0      14
 102.0      14
 101.0      12
 103.0       8
 104.0       7
 109.0       6
 107.0       6
-1.0         6
 108.0       5
 110.0       4
 105.0       2
-177.0       1
-40.0        1
-47.0        1
-49.0        1
 106.0       1
Name: AGE, dtype: int64

In [45]:
people_df['PERSON_TYPE'].value_counts()

DRIVER                 951888
PASSENGER              250405
PEDESTRIAN              14000
BICYCLE                  8543
NON-MOTOR VEHICLE        1055
NON-CONTACT VEHICLE       221
Name: PERSON_TYPE, dtype: int64

KeyError: 'PERSON_TYPE'